In [1]:
import re

def trimming(file_name):
    # 1. 역순으로 제일 먼저 찾은 이메일 밑으로 다 없앰
    # 2. 나머지 특수문자들 다 없앰 (. 빼고)
    # 3. 공백문자열 다 없앰
    
    with open(file_name,'r',encoding='utf8') as f:
        articles = f.readlines()
    
    # 1.
    is_email = []
    for each in articles:
        tmp = re.findall('[^@]+@[^@]+\.[^@]+', each)
        is_email.append(tmp)
    
    is_email = [True if len(each)!=0 else False for each in is_email]
    upper_bound = len(is_email) - 2 - is_email[::-1].index(True) if True in is_email else len(is_email)-1
    
    articles = [articles[idx] if not is_email[idx] else '' for idx in range(0,upper_bound)]
    
    # 2.
    for idx, val in enumerate(articles):
        converted = re.sub('[^가-힣0-9a-zA-Z.\\s]', ' ', val)
        articles[idx] = converted
    
    # 3.
    articles = [each for each in articles if each != '']
    
    return articles

In [2]:
# file_name = '정치/0-2018-04-14 17-03.txt'
# trimmed = trimming(file_name)
# trimmed = ''.join(trimmed)

In [3]:
# with open(file_name,'w',encoding='utf-8') as f:
#     f.write(trimmed)

In [4]:
# from os import listdir
# from os.path import isfile, join
# path = '정치'

# file_names = [f for f in listdir(path) if isfile(join(path, f)) and  f.endswith(".txt")]
# full_names = ['{}/{}'.format(path, each) for each in file_names ]

In [5]:
from datetime import datetime

now = datetime.now()
cur_time = now.strftime('%Y-%m-%d %H-%M-%S')

In [6]:
from os import makedirs as mk
from os import path as pt

path = ['정치','경제','사회','세계','생활문화','IT과학']

new_path = '{}/{}'.format('preprocess',cur_time)

if not pt.exists(new_path):
    mk(new_path)
    for area in path:
        mk('{}/{}'.format(new_path,area))

In [8]:
from os import listdir
from os.path import isfile, join

for each in path:
    crawling_path = '{}/{}'.format('crawling', each)
    file_names = [f for f in listdir(crawling_path) if isfile(
        join(crawling_path, f)) and f.endswith(".txt")]
    # 파일이름

    full_names = ['{}/{}'.format(each, tmp) for tmp in file_names]
    # 영역/파일이름

    crawling_path = ['{}/{}'.format('crawling',tmp) for tmp in full_names]
    # crawling/영역/파일이름
    
    for idx, file_name in enumerate(crawling_path):
        trimmed = trimming(file_name)
        trimmed = ''.join(trimmed)
        
        dest_path = '{}/{}/{}'.format('preprocess',cur_time,full_names[idx])

        with open(dest_path, 'w', encoding='utf-8') as f:
            f.write(trimmed)

    print(each, ' completed !')

정치  completed !
경제  completed !
사회  completed !
세계  completed !
생활문화  completed !
IT과학  completed !


In [11]:
import pickle
import pandas as pd

In [12]:
path = [('정치',0), ('경제',1), ('사회',2), ('세계',3), ('생활문화',4), ('IT과학',5)]
# path = [('정치',0)]

total_df = pd.DataFrame(columns=['content','area'])

for each in path:
    preprocess_path = '{}/{}/{}'.format('preprocess',cur_time, each[0])
    file_names = [f for f in listdir(preprocess_path) if isfile(
        join(preprocess_path, f)) and f.endswith(".txt")]
    full_names = ['{}/{}'.format(preprocess_path, tmp) for tmp in file_names]

    print(full_names)
    
#     contents = []
#     written_date = []
    
#     df = pd.DataFrame(columns=['content','area'])
    
#     for file_name in full_names:

#         with open(file_name, 'r', encoding='utf-8') as f:
#             contents.append(f.read())
#             written_date.append(file_name)
            
#     df['content'] = pd.Series(contents)
#     df['area'] = each[1]
#     df['written_date'] = pd.Series(written_date)
#     df['written_date'] = df['written_date'].apply(lambda x: '-'.join(x.split('/')[1].replace('.txt','').split('-')[1:]))
#     df['written_date'] = df['written_date'].apply(lambda x: '-'.join(x.split()))
#     df['title'] = df['content'].apply(lambda x: x.split('\n')[0].replace('\t',''))
    
#     df = df[['title','content','written_date','area']]
    
#     total_df = total_df.append(df)
#     print(each,' completed !')

# total_df['area'] = total_df.area.astype('int64')
# total_df.reset_index(drop=True, inplace=True)

['preprocess/2018-04-19 20-02-13/정치/0-2018-04-14 17-03.txt', 'preprocess/2018-04-19 20-02-13/정치/0-2018-04-19 02-31.txt', 'preprocess/2018-04-19 20-02-13/정치/1-2018-04-14 17-00.txt', 'preprocess/2018-04-19 20-02-13/정치/1-2018-04-19 02-01.txt', 'preprocess/2018-04-19 20-02-13/정치/10-2018-04-14 15-32.txt', 'preprocess/2018-04-19 20-02-13/정치/10-2018-04-18 23-53.txt', 'preprocess/2018-04-19 20-02-13/정치/100-2018-04-14 05-04.txt', 'preprocess/2018-04-19 20-02-13/정치/100-2018-04-18 19-15.txt', 'preprocess/2018-04-19 20-02-13/정치/1000-2018-04-18 11-41.txt', 'preprocess/2018-04-19 20-02-13/정치/1001-2018-04-18 11-41.txt', 'preprocess/2018-04-19 20-02-13/정치/1002-2018-04-18 11-40.txt', 'preprocess/2018-04-19 20-02-13/정치/1003-2018-04-18 11-40.txt', 'preprocess/2018-04-19 20-02-13/정치/1004-2018-04-18 17-44.txt', 'preprocess/2018-04-19 20-02-13/정치/1005-2018-04-18 17-44.txt', 'preprocess/2018-04-19 20-02-13/정치/1006-2018-04-18 17-41.txt', 'preprocess/2018-04-19 20-02-13/정치/1007-2018-04-18 17-40.txt', 'preproce

['preprocess/2018-04-19 20-02-13/사회/0-2018-04-14 18-04.txt', 'preprocess/2018-04-19 20-02-13/사회/0-2018-04-19 03-07.txt', 'preprocess/2018-04-19 20-02-13/사회/1-2018-04-14 18-04.txt', 'preprocess/2018-04-19 20-02-13/사회/1-2018-04-19 03-03.txt', 'preprocess/2018-04-19 20-02-13/사회/10-2018-04-14 16-14.txt', 'preprocess/2018-04-19 20-02-13/사회/10-2018-04-19 01-11.txt', 'preprocess/2018-04-19 20-02-13/사회/100-2018-04-14 09-00.txt', 'preprocess/2018-04-19 20-02-13/사회/100-2018-04-18 20-09.txt', 'preprocess/2018-04-19 20-02-13/사회/1000-2018-04-18 17-28.txt', 'preprocess/2018-04-19 20-02-13/사회/1001-2018-04-18 17-28.txt', 'preprocess/2018-04-19 20-02-13/사회/1002-2018-04-18 17-28.txt', 'preprocess/2018-04-19 20-02-13/사회/1003-2018-04-18 17-28.txt', 'preprocess/2018-04-19 20-02-13/사회/1004-2018-04-18 17-28.txt', 'preprocess/2018-04-19 20-02-13/사회/1005-2018-04-18 17-27.txt', 'preprocess/2018-04-19 20-02-13/사회/1006-2018-04-18 17-27.txt', 'preprocess/2018-04-19 20-02-13/사회/1007-2018-04-18 17-25.txt', 'preproce

['preprocess/2018-04-19 20-02-13/생활문화/0-2018-04-14 16-05.txt', 'preprocess/2018-04-19 20-02-13/생활문화/0-2018-04-19 00-46.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1-2018-04-14 15-53.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1-2018-04-19 00-45.txt', 'preprocess/2018-04-19 20-02-13/생활문화/10-2018-04-14 13-13.txt', 'preprocess/2018-04-19 20-02-13/생활문화/10-2018-04-18 21-16.txt', 'preprocess/2018-04-19 20-02-13/생활문화/100-2018-04-14 02-02.txt', 'preprocess/2018-04-19 20-02-13/생활문화/100-2018-04-18 12-01.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1000-2018-04-18 06-03.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1001-2018-04-18 06-02.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1002-2018-04-18 06-02.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1003-2018-04-18 06-02.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1004-2018-04-18 06-01.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1005-2018-04-18 05-56.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1006-2018-04-18 05-40.txt', 'preprocess/2018-04-19 20-02-13/생활문화/1007-

In [ ]:
with open('dataset/training_{}.pkl'.format(cur_time),'wb') as f:
    pickle.dump(total_df, f)